In [72]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image

import matplotlib.pyplot as plt

In [73]:
def one_hot_encoding(n, length):
    one_hot = [0] * length
    one_hot[n] = 1
    
    return one_hot

In [91]:
def image_to_array(data, p):
    arr = np.empty([0, 128, 128, 3])
    label = [one_hot_encoding(p, 2) for _ in range(len(data))]
    for i in range(len(data)):
        image = Image.open(data[i])
        image = image.resize((128, 128))
        numpy_image = np.array(image)
        arr = np.append(arr,[numpy_image],axis = 0)
    label = np.array(label)
    return arr, label

In [83]:
train_path = r"D:\cctv_pjt\training_set\training_set"
test_path = r"D:\cctv_pjt\test_set\test_set"
train_cat = glob.glob(train_path + '\cats' + '\\'+ "*.jpg")
train_dog = glob.glob(train_path + '\dogs' + '\\'+ "*.jpg")
test_cat = glob.glob(test_path + '\cats' + '\\'+ "*.jpg")
test_dog = glob.glob(test_path + '\dogs' + '\\'+ "*.jpg")

In [92]:
list = [train_cat, train_dog, test_cat, test_dog]
datasets = []
labels = []
for i, lst in enumerate(list):
    tmp_data, tmp_label = image_to_array(train_cat, i%2)
    datasets.append(tmp_data)
    labels.append(tmp_label)

In [121]:
datasets[0].shape

(4000, 128, 128, 3)

In [122]:
labels[0].shape

(4000, 2)

In [ ]:
train_data = np.concatenate((datasets[0], datasets[1]), axis = 0)
train_data = train_data/255

train_label = np.concatenate((labels[0], labels[1]), axis = 0)
train_label.shape

In [126]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(layers.Conv2D(4, 4, strides=2, activation='relu', padding='same', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(8, 4, activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(4, 3, activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(8, 3, activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_76 (Conv2D)           (None, 64, 64, 4)         196       
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 32, 32, 8)         520       
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 16, 16, 4)         292       
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 8, 8, 4)           0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 8, 8, 4)         

In [127]:
from tensorflow.keras import optimizers

model.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [128]:
history = model.fit(train_data, 
                    train_label,
                    validation_split = 0.1,
                    shuffle=True,
                    epochs=10,
                    batch_size=10)

Epoch 1/10
720/720 [==============================] - 12s 16ms/step - loss: 0.6971 - accuracy: 0.5378 - val_loss: 0.8454 - val_accuracy: 0.0000e+00
Epoch 2/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6886 - accuracy: 0.5517 - val_loss: 0.8224 - val_accuracy: 0.0000e+00
Epoch 3/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6870 - accuracy: 0.5591 - val_loss: 0.8057 - val_accuracy: 0.0000e+00
Epoch 4/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6873 - accuracy: 0.5560 - val_loss: 0.8534 - val_accuracy: 0.0000e+00
Epoch 5/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6898 - accuracy: 0.5468 - val_loss: 0.8044 - val_accuracy: 0.0000e+00
Epoch 6/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6874 - accuracy: 0.5536 - val_loss: 0.8250 - val_accuracy: 0.0000e+00
Epoch 7/10
720/720 [==============================] - 11s 15ms/step - loss: 0.6887 - accuracy: 0.5499 - val_loss

In [129]:
datasets[2] = datasets[2]/255
datasets[3] = datasets[3]/255
test_data = np.concatenate((datasets[2], datasets[3]), axis = 0)

test_label = np.concatenate((labels[2], labels[3]), axis = 0)
test_label.shape

(8000, 2)

In [130]:
print("{:.4f}".format(model.evaluate(datasets[2], labels[2])[1]))
print("{:.4f}".format(model.evaluate(datasets[3], labels[3])[1]))
print("{:.4f}".format(model.evaluate(test_data, test_label)[1]))

125/125 [==============================] - 2s 14ms/step - loss: 0.5928 - accuracy: 1.0000
1.0000
125/125 [==============================] - 2s 17ms/step - loss: 0.8047 - accuracy: 0.0000e+00
0.0000
250/250 [==============================] - 3s 12ms/step - loss: 0.6987 - accuracy: 0.5000
0.5000
